In [1]:
import time
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from models import *
from datasets import *
from utils import *
from nltk.translate.bleu_score import corpus_bleu
from nltk import pos_tag
import pandas as pd
import pickle


###################################################################################################
#For different Experiments we will use different value of the net here
# net = [lenet, vgg16, resnet18, resnet34, resnet50, resnet101,resnet18skip, 
#            resnet18Mid, resnet34Mid, resnet50Mid, resnet101Mid]
net = 'resnet18skip'

emb_dim = 512  # Default dimension of word embeddings
pretrained = False #set to true for using pretrained embeddings
if(pretrained):
    emb_dim = 300
    
# Data parameters
data_folder = '/datasets/home/home-02/57/257/asakhale/ICProject/data_generated'  # folder with data files saved by create_input_files.py
data_name = 'coco_5_cap_per_img_5_min_word_freq'  # base name shared by data files
# Path to save logs
csv_path ='/datasets/home/50/650/agokhale/sgr_again/logs/new_loss_pretrained_18_exp_logs.csv'
# Path to use pretrained GloVe embeddings
pretrained_emb_path = '/datasets/home/50/650/agokhale/sgr_again/embed/6B.300_embed_init.pkl'
####################################################################################################

NEW_LOSS_WEIGHT = 0.1

# Data parameters
data_folder = '/datasets/home/home-02/57/257/asakhale/ICProject/data_generated'  # folder with data files saved by create_input_files.py
data_name = 'coco_5_cap_per_img_5_min_word_freq'  # base name shared by data files

# Model parameters
feature_dim = 14
emb_dim = 512  # dimension of word embeddings
attention_dim = 512  # dimension of attention linear layers
decoder_dim = 512  # dimension of decoder RNN
dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
cudnn.benchmark = True  # set to true only if inputs to model are fixed size; otherwise lot of computational overhead

# Training parameters
start_epoch = 0
epochs = 5  # number of epochs to train for (if early stopping is not triggered)
epochs_since_improvement = 0  # keeps track of number of epochs since there's been an improvement in validation BLEU
batch_size = 80
workers = 1  # for data-loading; right now, only 1 works with h5py
encoder_lr = 1e-4  # learning rate for encoder if fine-tuning
decoder_lr = 4e-4  # learning rate for decoder
grad_clip = 5.  # clip gradients at an absolute value of
alpha_c = 1.  # regularization parameter for 'doubly stochastic attention', as in the paper
best_bleu4 = 0.  # BLEU-4 score right now
print_freq = 100  # print training/validation stats every __ batches
fine_tune_encoder = False  # fine-tune encoder?
checkpoint = None#'checkpoint_coco_5_cap_per_img_5_min_word_freq.pth.tar'# path to checkpoint, None if none

csv_path ='/datasets/home/home-02/57/257/asakhale/ICProject/logs/newloss_pre_18_exp_logs.csv'
pretrained = False
pretrained_emb_path= '/datasets/home/home-02/57/257/asakhale/ICProject/embed/6B.300_embed_init.pkl'
embedding = []
# Read word map
word_map_file = os.path.join(data_folder, 'WORDMAP_' + data_name + '.json')
inv_map = []
decoder = []
with open(word_map_file, 'r') as j:
    word_map = json.load(j)
inv_map = [k for k, v in word_map.items()]

df = pd.DataFrame(columns= ['epoch', 'trloss', 'tracc5', 'val_loss', 'val_acc5', 'recentbleu4', 'bestbleu4', 'epochs_since_imp', 'fine_tune_enc'])


In [2]:
def train(train_loader, encoder, decoder, criterion,criterion2, encoder_optimizer, decoder_optimizer, epoch):
    """
    Performs one epoch's training.

    :param train_loader: DataLoader for training data
    :param encoder: encoder model
    :param decoder: decoder model
    :param criterion: loss layer
    :param encoder_optimizer: optimizer to update encoder's weights (if fine-tuning)
    :param decoder_optimizer: optimizer to update decoder's weights
    :param epoch: epoch number
    """

    decoder.train()  # train mode (dropout and batchnorm is used)
    encoder.train()

    batch_time = AverageMeter()  # forward prop. + back prop. time
    data_time = AverageMeter()  # data loading time
    losses = AverageMeter()  # loss (per word decoded)
    top5accs = AverageMeter()  # top5 accuracy

    start = time.time()

    # Batches
    for i, (imgs, caps, caplens) in enumerate(train_loader):
        data_time.update(time.time() - start)

        # Move to GPU, if available
        imgs = imgs.to(device)
        caps = caps.to(device)
        caplens = caplens.to(device)

        # Forward prop.
        imgs = encoder(imgs)
        scores, caps_sorted, decode_lengths, alphas, sort_ind = decoder(imgs, caps, caplens)

        # Since we decoded starting with <start>, the targets are all words after <start>, up to <end>
        targets = caps_sorted[:, 1:]
#         print(scores.shape, targets.shape)

        #insert new loss here
#         print('computing Loss 2')
        
        pred_noun_idxs, targ_noun_idxs, dec_lens = pos_loss_pairs(scores, targets)

#         print('noun idxs = {}, targ noun idx = {}'.format(pred_noun_idxs.shape, targ_noun_idxs.shape))
        # Remove timesteps that we didn't decode at, or are pads
        # pack_padded_sequence is an easy trick to do this
        scores = pack_padded_sequence(scores, decode_lengths, batch_first=True).data
        targets = pack_padded_sequence(targets, decode_lengths, batch_first=True).data
        
        print('SHAPE OF SCORES : ', scores.shape)
        # Calculate loss
        loss2 = 0
        
        if len(dec_lens) > 0:
            pred_noun_idxs = pred_noun_idxs.to(device)
            targ_noun_idxs = targ_noun_idxs.to(device)
            scores_noun = pack_padded_sequence(pred_noun_idxs, dec_lens, batch_first=True).data
            targets_noun = pack_padded_sequence(targ_noun_idxs, dec_lens, batch_first=True).data
            
            loss2 = criterion2(scores_noun, targets_noun)
            loss2.to(device)
        
        loss = criterion(scores, targets)
        
        
        loss.to(device)
        # Add doubly stochastic attention regularization
        loss = loss + NEW_LOSS_WEIGHT * loss2 + alpha_c * ((1. - alphas.sum(dim=1)) ** 2).mean()
        # Back prop.
        decoder_optimizer.zero_grad()
        if encoder_optimizer is not None:
            encoder_optimizer.zero_grad()
        loss.backward()

        # Clip gradients
        if grad_clip is not None:
            clip_gradient(decoder_optimizer, grad_clip)
            if encoder_optimizer is not None:
                clip_gradient(encoder_optimizer, grad_clip)

        # Update weights
        decoder_optimizer.step()
        if encoder_optimizer is not None:
            encoder_optimizer.step()

        # Keep track of metrics
        top5 = accuracy(scores, targets, 5)
        losses.update(loss.item(), sum(decode_lengths))
        top5accs.update(top5, sum(decode_lengths))
        batch_time.update(time.time() - start)

        start = time.time()

        # Print status
        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data Load Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Top-5 Accuracy {top5.val:.3f} ({top5.avg:.3f})'.format(epoch, i, len(train_loader),
                                                                          batch_time=batch_time,
                                                                          data_time=data_time, loss=losses,
                                                                          top5=top5accs))


    return loss, top5

In [3]:
def validate(val_loader, encoder, decoder, criterion):
    """
    Performs one epoch's validation.

    :param val_loader: DataLoader for validation data.
    :param encoder: encoder model
    :param decoder: decoder model
    :param criterion: loss layer
    :return: BLEU-4 score
    """
    decoder.eval()  # eval mode (no dropout or batchnorm)
    if encoder is not None:
        encoder.eval()

    batch_time = AverageMeter()
    losses = AverageMeter()
    top5accs = AverageMeter()

    start = time.time()

    references = list()  # references (true captions) for calculating BLEU-4 score
    hypotheses = list()  # hypotheses (predictions)

    # explicitly disable gradient calculation to avoid CUDA memory error
    # solves the issue #57
    with torch.no_grad():
        # Batches
        for i, (imgs, caps, caplens, allcaps) in enumerate(val_loader):

            # Move to device, if available
            imgs = imgs.to(device)
            caps = caps.to(device)
            caplens = caplens.to(device)

            # Forward prop.
            if encoder is not None:
                imgs = encoder(imgs)
            scores, caps_sorted, decode_lengths, alphas, sort_ind = decoder(imgs, caps, caplens)

            # Since we decoded starting with <start>, the targets are all words after <start>, up to <end>
            targets = caps_sorted[:, 1:]

            # Remove timesteps that we didn't decode at, or are pads
            # pack_padded_sequence is an easy trick to do this
            scores_copy = scores.clone()
            scores= pack_padded_sequence(scores, decode_lengths, batch_first=True).data
            targets = pack_padded_sequence(targets, decode_lengths, batch_first=True).data

            # Calculate loss
            loss = criterion(scores, targets)

            # Add doubly stochastic attention regularization
            loss += alpha_c * ((1. - alphas.sum(dim=1)) ** 2).mean()

            # Keep track of metrics
            losses.update(loss.item(), sum(decode_lengths))
            top5 = accuracy(scores, targets, 5)
            top5accs.update(top5, sum(decode_lengths))
            batch_time.update(time.time() - start)

            start = time.time()

            if i % print_freq == 0:
                print('Validation: [{0}/{1}]\t'
                      'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Top-5 Accuracy {top5.val:.3f} ({top5.avg:.3f})\t'.format(i, len(val_loader), batch_time=batch_time,
                                                                                loss=losses, top5=top5accs))

            # Store references (true captions), and hypothesis (prediction) for each image
            # If for n images, we have n hypotheses, and references a, b, c... for each image, we need -
            # references = [[ref1a, ref1b, ref1c], [ref2a, ref2b], ...], hypotheses = [hyp1, hyp2, ...]

            # References
            allcaps = allcaps[sort_ind]  # because images were sorted in the decoder
            for j in range(allcaps.shape[0]):
                img_caps = allcaps[j].tolist()
                img_captions = list(
                    map(lambda c: [w for w in c if w not in {word_map['<start>'], word_map['<pad>']}],
                        img_caps))  # remove <start> and pads
                references.append(img_captions)

            # Hypotheses
            _, preds = torch.max(scores_copy, dim=2)
            preds = preds.tolist()
            temp_preds = list()
            for j, p in enumerate(preds):
                temp_preds.append(preds[j][:decode_lengths[j]])  # remove pads
            preds = temp_preds
            hypotheses.extend(preds)

            assert len(references) == len(hypotheses)

        # Calculate BLEU-4 scores
        bleu4 = corpus_bleu(references, hypotheses)

        print(
            '\n * LOSS - {loss.avg:.3f}, TOP-5 ACCURACY - {top5.avg:.3f}, BLEU-4 - {bleu}\n'.format(
                loss=losses,
                top5=top5accs,
                bleu=bleu4))

    return bleu4, loss, top5

In [4]:
def pos_loss_pairs(predictions_onehot, targets):
    global embedding
    #accepts one-hot encodings of the predictions and the targets
    #converts to words, detects POS nouns using nltk
    #converts to embeddings, matches most similar nouns
    #cross entropy per word weighed by the similarity measure between the noun predicted and the target noun
    predictions_onehot = predictions_onehot.cpu()
    print(predictions_onehot.shape)
    predictions = torch.argmax(predictions_onehot, dim = 2)
#     print(predictions.shape)
    targets = targets[:,:predictions.shape[1]]
    pred_s = []
    targ_s = []
    max_pred_idx_len = 0
    pred_idx = []
    targ_idx = []
    for i in range(predictions.shape[0]):
        p_sent =[]
        p_idx = []
        t_sent = []
        t_idx =[]
        for j in range(predictions.shape[1]):
            word = (inv_map[predictions[i][j]-1])
            #track the index of the word
            if(pos_tag([word])[-1][1] == 'NN'):
                p_idx.append(j)
            #store the mapping of the word
#             p_sent.append(inv_map[predictions[i][j]-1])
            
            word = (inv_map[targets[i][j]-1])
            #track the index of the word
            if(pos_tag([word])[-1][1] == 'NN'):
                t_idx.append(j)
            #store the mapping of the word
#             t_sent.append(targets[i][j])
        if(len(p_idx) > max_pred_idx_len):
            max_pred_idx_len = len(p_idx)
        pred_idx.append(p_idx)
    
        targ_idx.append(t_idx)

    #keep track of the original positions of the words, and then reorder preds and tgts accordingly
    #pull out those indices per predicted and target sentence and get embeddings of each word

#     print(pos_tag([inv_map[predictions[0,pred_idx[0][0]]-1]]))
    max_matches = []
    max_n_nouns = 0
    decode_lengths = []
    if(len(targ_idx) > 0 and len(pred_idx) > 0):
#now we pull out predictions and targets at positions stored in pred_idx and targ_idx and then match them to compute the loss
        for i in range(len(pred_idx)):
            pred_emb_getter = torch.Tensor(pred_idx[i]).long().to(device)
            targ_emb_getter = torch.Tensor(targ_idx[i]).long().to(device)

            p_embed = embeddings(pred_emb_getter)
            t_embed = embeddings(targ_emb_getter)
    #         print(p_embed.shape)

            p_norm = torch.norm(p_embed, p = 2, dim = 1)
            t_norm = torch.norm(t_embed, p = 2, dim = 1)
    #         print(p_norm.shape)

            p_embed = p_embed.t() * 1. / p_norm
            t_embed = t_embed.t() * 1. / t_norm

    #         print(p_embed.shape)
            print('Shape of Predicted Noun embeddings : ',p_embed.shape)
            print('Shape of Target Noun embeddings : ',t_embed.shape)
            sim_mtx = torch.mm(p_embed.t(), t_embed)
            print('Shape of Similarity Matrix : ',sim_mtx.shape)
    #         print(sim_mtx.shape)
            max_sim_per_pred_noun = torch.argmax(sim_mtx, dim = 1)
    #         print(max_sim_per_pred_noun)

            #pull out the words from vocab tensor
            max_match_in_sentence = [targ_emb_getter[i] for i in max_sim_per_pred_noun]
            decode_lengths.append(len(max_match_in_sentence))
            arr = np.zeros(51)
            arr[:len(max_match_in_sentence)] = max_match_in_sentence
            max_matches.append(arr)

#             if(len(max_match_in_sentence) > max_n_nouns):
#                 max_n_nouns = len(max_match_in_sentence)

            #pick out indices of the max valued

    #         for j in range(len(pred_idx[i])):
    #             print(pos_tag([inv_map[predictions[0,pred_idx[i][j]]-1]]))
    ret_pred_idx = []
    ret_pred_idx = torch.zeros((len(pred_idx), max_pred_idx_len, 9490))
    for i in range(len(pred_idx)):
        p_idx = pred_idx[i]
        arr = torch.zeros((max_pred_idx_len, 9490))
        arr[:len(p_idx),:] = predictions_onehot[i, p_idx,:]
        ret_pred_idx[i] = arr
    decode_lengths.sort(reverse = True)
    max_matches = torch.Tensor(np.array(max_matches)).long()
    return ret_pred_idx, max_matches, decode_lengths


In [5]:


if checkpoint is None:
    if(net=='lenet'):
        encoder = EncoderWithLeNet(encoded_image_size = feature_dim)
    elif(net=='resnet18skip'):
        encoder = EncoderWithSkip(encoded_image_size = feature_dim)
    else:
        encoder = Encoder(encoded_image_size = feature_dim, model = net)
    encoder.fine_tune(fine_tune_encoder)
    encoder_optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, encoder.parameters()),
                                         lr=encoder_lr) if fine_tune_encoder else None
    encoder_out_dim = encoder.get_out_dim()

    decoder = DecoderWithAttention(attention_dim=attention_dim,
                                   embed_dim=emb_dim,
                                   encoder_dim = encoder_out_dim,
                                   decoder_dim=decoder_dim,
                                   vocab_size=len(word_map),
                                   dropout=dropout)
    decoder_optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, decoder.parameters()),
                                         lr=decoder_lr)
    if(pretrained==True):
        pretrained_embeddings = torch.Tensor(pickle.load(open(pretrained_emb_path, 'rb')))
        decoder.load_pretrained_embeddings(pretrained_embeddings)  # pretrained_embeddings should be of dimensions (len(word_map), emb_dim)      
        decoder.fine_tune_embeddings(True)

else:
    df = pd.read_csv(csv_path)
    checkpoint = torch.load(checkpoint)
    start_epoch = checkpoint['epoch'] + 1
    epochs_since_improvement = checkpoint['epochs_since_improvement']
    best_bleu4 = checkpoint['bleu-4']
    decoder = checkpoint['decoder']
    decoder_optimizer = checkpoint['decoder_optimizer']
    encoder = checkpoint['encoder']
    encoder_optimizer = checkpoint['encoder_optimizer']
    if fine_tune_encoder is True and encoder_optimizer is None:
        encoder.fine_tune(fine_tune_encoder)
        encoder_optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, encoder.parameters()),
                                             lr=encoder_lr)


# Move to GPU, if available
decoder = decoder.to(device)
encoder = encoder.to(device)

embedding = decoder.get_embedding()

# Loss function
criterion = nn.CrossEntropyLoss().to(device)
criterion2 = nn.CrossEntropyLoss().to(device)

# Custom dataloaders
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
train_loader = torch.utils.data.DataLoader(CaptionDataset(data_folder, data_name, 'TRAIN', transform=transforms.Compose([normalize])),
batch_size=batch_size, shuffle=True, num_workers=workers, pin_memory=True)
val_loader = torch.utils.data.DataLoader(CaptionDataset(data_folder, data_name, 'VAL', transform=transforms.Compose([normalize])),
batch_size=batch_size, shuffle=True, num_workers=workers, pin_memory=True)

# Epochs
train_loss = 0 
tr_acc5 = 0
val_loss = 0 
val_acc5 = 0
stat_list = []
df_new = pd.DataFrame()
for epoch in range(start_epoch, epochs):
    if epochs_since_improvement == 20:
        break
    if epochs_since_improvement > 0 and epochs_since_improvement % 8 == 0:
        adjust_learning_rate(decoder_optimizer, 0.8)
        if fine_tune_encoder:
            adjust_learning_rate(encoder_optimizer, 0.8)

    # One epoch's training
    train_loss, tr_acc5 = train(train_loader=train_loader,
          encoder=encoder,
          decoder=decoder,
          criterion=criterion,
          criterion2=criterion2,
          encoder_optimizer=encoder_optimizer,
          decoder_optimizer=decoder_optimizer,
          epoch=epoch)
     # One epoch's validation
    recent_bleu4, val_loss, val_acc5 = validate(val_loader=val_loader,
                            encoder=encoder,
                            decoder=decoder,
                            criterion=criterion)

    # Check if there was an improvement
    is_best = recent_bleu4 > best_bleu4
    best_bleu4 = max(recent_bleu4, best_bleu4)
    if not is_best:
        epochs_since_improvement += 1
        print("\nEpochs since last improvement: %d\n" % (epochs_since_improvement,))
    else:
        epochs_since_improvement = 0
#global best_bleu4, epochs_since_improvement, checkpoint, start_epoch, fine_tune_encoder, data_name, word_map

    stat_list.append([epoch, train_loss, tr_acc5, val_loss, val_acc5, recent_bleu4, best_bleu4, epochs_since_improvement, fine_tune_encoder])
    # Save checkpoint
    save_checkpoint('no_embed_res18_2nd_run'+data_name, epoch, epochs_since_improvement, encoder, decoder, encoder_optimizer,
                    decoder_optimizer, recent_bleu4, is_best)

    df_new = pd.DataFrame(stat_list,columns= ['epoch', 'trloss', 'tracc5', 'val_loss', 'val_acc5', 'recentbleu4', 'bestbleu4', 'epochs_since_imp', 'fine_tune_enc'])
    df = pd.concat([df, df_new], ignore_index = True)
    df.to_csv(csv_path, index = False)

torch.Size([80, 19, 9490])


LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger/averaged_perceptron_tagger.pickle[0m

  Searched in:
    - '/datasets/home/57/257/asakhale/nltk_data'
    - '/opt/conda/nltk_data'
    - '/opt/conda/share/nltk_data'
    - '/opt/conda/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
